In [ ]:
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [1]:
from unsloth import FastLanguageModel
import torch

In [2]:
max_seq_length = 2048
# max_seq_length = 1024
dtype = None

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit",
    # model_name = "unsloth/gemma-7b-bnb-4bit",
    model_name = "unsloth/mistral-7b-bnb-4bit",
    # model_name = "unsloth/tinyllama-bnb-4bit", # "unsloth/tinyllama" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 3080 Laptop GPU. Max memory: 8.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


2024-04-22 14:02:22.407979: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 14:02:22.463898: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 14:02:22.463934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 14:02:22.468733: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 14:02:22.499649: I tensorflow/core/platform/cpu_feature_guar

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    # lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["q"]
    outputs      = examples["a"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = tokenizer.apply_chat_template([{"role": "user", "content": instruction},{"role": "assistant", "content":output}], tokenize=False) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("pythainlp/han-instruct-dataset-v2.0", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = None,
        num_train_epochs=3,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [7]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3080 Laptop GPU. Max memory = 8.0 GB.
4.5 GB of memory reserved.


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 435 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 162
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.669400
2,1.771700
3,1.476400
4,1.446600
5,1.321400
6,1.269100
7,1.361900
8,1.277600
9,1.142000
10,1.011400


In [9]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4469.1217 seconds used for training.
74.49 minutes used for training.
Peak reserved memory = 5.662 GB.
Peak reserved memory for training = 1.162 GB.
Peak reserved memory % of max memory = 70.775 %.
Peak reserved memory for training % of max memory = 14.525 %.


In [15]:
FastLanguageModel.for_inference(model)

In [16]:
inputs = tokenizer([tokenizer.apply_chat_template([{"role": "user", "content": "ios กับ android อะไรดีกว่ากัน"}], tokenize=False)], return_tensors = "pt").to("cuda")


outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s><s> [INST] ios กับ android อะไรดีกว่ากัน [/INST] ขึ้นอยู่กับความต้องการของคุณ หากคุณต้องการเครื่องมือที่มีความปลอดภัยและไม่มีการแบ่งปันข้อมูล ให้เลือก ios เนื่องจาก ios มีความปลอดภัยมากกว่า android แต่หากคุณต้องการเครื่องมือที่มีความสะดวกและสะดวก ให้เลือก android เนื่องจาก android มีความสะดวกกว่า ios </s>']

In [17]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer([tokenizer.apply_chat_template([{"role": "user", "content": "อยากสูงต้องทำยังไง"}], tokenize=False)], return_tensors = "pt").to("cuda")


outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s><s> [INST] อยากสูงต้องทำยังไง [/INST] คุณสามารถเพิ่มความสูงของตัวคุณได้หลายวิธี เช่น การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถในการเดิน การทำกิจกรรมออกdoors เพื่อเพิ่มความสามารถใน

In [18]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer([tokenizer.apply_chat_template([{"role": "user", "content": "ใครเป็นนายกของประเทศไทย"}], tokenize=False)], return_tensors = "pt").to("cuda")


outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s><s> [INST] ใครเป็นนายกของประเทศไทย [/INST] นายกรัฐมนตรีของประเทศไทย ปัจจุบันคือ นายกรัฐมนตรี พล.สก. ประยุทธ์ ชายากร </s>']